# create or update environment dependencies

In [ ]:
#!conda env create -f environment.yaml
#!conda env update --name lost --file environment.yaml --prune

# import libraries

In [ ]:
import sys
import xarray as xr
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt

# get ERA5 data

In [ ]:
sys.path.append('/Users/backeb/Documents/repos/myscripts/era5')
from download_era5 import get_era5_single_levels


get_era5_single_levels( 
    longitude_min = 45.0,
    longitude_max = 65.0,
    latitude_min = -50.0,
    latitude_max = -30.0,
    date_min = '2021-05-15',
    date_max = '2021-06-15',
    vars = ('10m_u_component_of_wind', '10m_v_component_of_wind'),
    outdir = '/Users/backeb/Documents/data/lost_use_case_wakashio_maru'
)

In [ ]:
ds = xr.open_dataset('/Users/backeb/Documents/data/lost_use_case_wakashio_maru/era5/era5.nc')
ds.latitude.data[-1:0:-1]

# get CMEMS data

In [ ]:
sys.path.append('/Users/backeb/Documents/repos/myscripts/cmems')
from download_cmems_physics import get_nrt_hourly

get_nrt_hourly(  
    username = 'bbackeberg', 
    password = 'iaTmwJ7D', 
    longitude_min = 45.0,
    longitude_max = 65.0,
    latitude_min = -50.0,
    latitude_max = -30.0,
    date_min = '2021-05-15',
    date_max = '2021-06-15',
    vars = ('uo', 'vo'),
    outdir = '/Users/backeb/Documents/data/lost_use_case_wakashio_maru'
)

In [ ]:
# ds = xr.open_dataset('/Users/backeb/Documents/data/lost_use_case_wakashio_maru/cmems/cmems.nc')
# ds.time.data

# interpolate to consistent timestamp

In [ ]:
ds_cmems = xr.open_dataset('/Users/backeb/Documents/data/lost_use_case_wakashio_maru/cmems/cmems.nc')
ds_era5 = xr.open_dataset('/Users/backeb/Documents/data/lost_use_case_wakashio_maru/era5/era5.nc')
ds_cmems_interp = ds_cmems.interp(time = ds_era5.time)



# check interpolation

In [ ]:
i=-1
f, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(5, 8), sharex=True)
ds_cmems.uo[i,0,:,:].plot(ax=ax1)
ax1.set_xlabel('')
ds_cmems_interp.uo[i,0,:,:].plot(ax=ax2)
ax2.set_xlabel('')
(ds_cmems.uo[i,0,:,:]-ds_cmems_interp.uo[i,0,:,:]).plot(ax=ax3)
ax3.set_title('difference')

# write to new netcdf

In [ ]:
ds_cmems_interp.to_netcdf('/Users/backeb/Documents/data/lost_use_case_wakashio_maru/cmems/cmems_interp.nc')

In [ ]:
ds_cmems_interp